# Goal:

For the sales, I'd like to denote whether it was sold at a loss or a gain with a red or green icon. The rest of the info could be put into a pop up box, if that works

oh and btw maybe put listings in blue or something so we can see that they are not sold yet

# Imports

In [1]:
import pandas as pd
import geopandas as gpd
import googlemaps
from shapely.geometry import Point
import os
import numpy as np
import re
from textwrap import shorten
from bs4 import BeautifulSoup

## Data read-in

In [2]:
# df = pd.read_clipboard()
# df.to_csv("new_sales_listings.csv")

In [3]:
df = pd.read_csv("new_sales_listings.csv")
df = df.drop(columns='Unnamed: 0')

In [4]:
df

,building name,address,seller,buyer,price,gain/loss for seller,type,color,story_link
0,Mondial River West,"910 West Huron Street, Chicago",Barings,Exposition Capital LLC,$42 million,loss,Sales,red,NaN
1,Residences at 8 East Huron,"8 East Huron Street, Chicago",CA Ventures,Dreyfuss Management,$61 million,gain,Sales,green,NaN
2,Haven on Long Grove Apartments,"826 Terrace Lake Drive, Aurora",Pensam Capital,Standard Real Estate Investments and the Vistr...,$94 million,gain,Sales,green,NaN
3,Germain House,"925 West Chicago Avenue, Chicago",Rockwell Property,Emmett Cunnigham Jr. and Sharon Shaw,$18 million,gain,Sales,green,NaN
4,The Aberdeen West Loop,"20 North Aberdeen Street, Chicago",Waterton,JAB Real Estate,$35 million,loss,Sales,red,NaN
5,Paragon Chicago,"1326 South Michigan Avenue, Chicago",John Murphy and CIM Group,FPA Multifamily,$144 million,loss,Sales,red,NaN
6,David Gassman's Northside portfolio,"6954 North Sheridan Road, Chicago*",David Gassman,multiple buyers,$72 million,NaN,Sales,green,https://therealdeal.com/chicago/2024/04/03/dav...
7,850 North Michigan,"850 North Michigan Avenue, Chicago",JPMorgan Asset Management,Crescent Heights,$80 million,loss,Sales,red,NaN
8,Sixteen30 Apartments,"14750 Wallin Drive, Plainfield",Wingspan Development Group,Continental Properties,$78 million,gain,Sales,green,NaN
9,Willow Crossing Apartments,"1031 Charlela Lane, Elk Grove Village",JPMorgan Asset Management,R.I.G. Capital,$89.3 million,gain,Sales,green,NaN


## Geocode

In [5]:
df['full_address'] = df['address'] + ' IL'

In [6]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [7]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'Full_Address' column and store the results in the 'geocoded' column
df['geocoded'] = df['full_address'].apply(geocode)

# Split 'geocoded' into 'lat' and 'lon', handling None values
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x is None else x).apply(pd.Series)

# Convert 'lat' and 'lon' to floats
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

# Create the 'geometry' column with Point objects
df['geometry'] = df.apply(lambda row: Point(row['lon'], row['lat']) if pd.notnull(row['lat']) and pd.notnull(row['lon']) else None, axis=1)

# Drop rows where geometry is None
df = df.dropna(subset=['geometry'])

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set CRS (Coordinate Reference System) to WGS 84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)


/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:131: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,building name,address,seller,buyer,price,gain/loss for seller,type,color,story_link,full_address,geocoded,lat,lon,geometry
0,Mondial River West,"910 West Huron Street, Chicago",Barings,Exposition Capital LLC,$42 million,loss,Sales,red,NaN,"910 West Huron Street, Chicago IL","(41.8953554, -87.6504573)",41.895355,-87.650457,POINT (-87.65046 41.89536)
1,Residences at 8 East Huron,"8 East Huron Street, Chicago",CA Ventures,Dreyfuss Management,$61 million,gain,Sales,green,NaN,"8 East Huron Street, Chicago IL","(41.8950168, -87.6277882)",41.895017,-87.627788,POINT (-87.62779 41.89502)
2,Haven on Long Grove Apartments,"826 Terrace Lake Drive, Aurora",Pensam Capital,Standard Real Estate Investments and the Vistr...,$94 million,gain,Sales,green,NaN,"826 Terrace Lake Drive, Aurora IL","(41.7367382, -88.2498821)",41.736738,-88.249882,POINT (-88.24988 41.73674)
3,Germain House,"925 West Chicago Avenue, Chicago",Rockwell Property,Emmett Cunnigham Jr. and Sharon Shaw,$18 million,gain,Sales,green,NaN,"925 West Chicago Avenue, Chicago IL","(41.8960354, -87.651032)",41.896035,-87.651032,POINT (-87.65103 41.89604)
4,The Aberdeen West Loop,"20 North Aberdeen Street, Chicago",Waterton,JAB Real Estate,$35 million,loss,Sales,red,NaN,"20 North Aberdeen Street, Chicago IL","(41.8823579, -87.65470839999999)",41.882358,-87.654708,POINT (-87.65471 41.88236)
5,Paragon Chicago,"1326 South Michigan Avenue, Chicago",John Murphy and CIM Group,FPA Multifamily,$144 million,loss,Sales,red,NaN,"1326 South Michigan Avenue, Chicago IL","(41.8649257, -87.62449289999999)",41.864926,-87.624493,POINT (-87.62449 41.86493)
6,David Gassman's Northside portfolio,"6954 North Sheridan Road, Chicago*",David Gassman,multiple buyers,$72 million,NaN,Sales,green,https://therealdeal.com/chicago/2024/04/03/dav...,"6954 North Sheridan Road, Chicago* IL","(42.00830029999999, -87.6622091)",42.008300,-87.662209,POINT (-87.66221 42.00830)
7,850 North Michigan,"850 North Michigan Avenue, Chicago",JPMorgan Asset Management,Crescent Heights,$80 million,loss,Sales,red,NaN,"850 North Michigan Avenue, Chicago IL","(41.8983126, -87.624648)",41.898313,-87.624648,POINT (-87.62465 41.89831)
8,Sixteen30 Apartments,"14750 Wallin Drive, Plainfield",Wingspan Development Group,Continental Properties,$78 million,gain,Sales,green,NaN,"14750 Wallin Drive, Plainfield IL","(41.6134211, -88.2199963)",41.613421,-88.219996,POINT (-88.22000 41.61342)
9,Willow Crossing Apartments,"1031 Charlela Lane, Elk Grove Village",JPMorgan Asset Management,R.I.G. Capital,$89.3 million,gain,Sales,green,NaN,"1031 Charlela Lane, Elk Grove Village IL","(41.9947864, -88.0276456)",41.994786,-88.027646,POINT (-88.02765 41.99479)


In [8]:
gdf.at[6,'Address']='<a href="https://therealdeal.com/chicago/2024/04/03/david-gassman-sells-chicago-apartment-portfolio-for-72m/"> 6954 North Sheridan Road, Chicago</a>'

In [9]:
gdf

,building name,address,seller,buyer,price,gain/loss for seller,type,color,story_link,full_address,geocoded,lat,lon,geometry,Address
0,Mondial River West,"910 West Huron Street, Chicago",Barings,Exposition Capital LLC,$42 million,loss,Sales,red,NaN,"910 West Huron Street, Chicago IL","(41.8953554, -87.6504573)",41.895355,-87.650457,POINT (-87.65046 41.89536),NaN
1,Residences at 8 East Huron,"8 East Huron Street, Chicago",CA Ventures,Dreyfuss Management,$61 million,gain,Sales,green,NaN,"8 East Huron Street, Chicago IL","(41.8950168, -87.6277882)",41.895017,-87.627788,POINT (-87.62779 41.89502),NaN
2,Haven on Long Grove Apartments,"826 Terrace Lake Drive, Aurora",Pensam Capital,Standard Real Estate Investments and the Vistr...,$94 million,gain,Sales,green,NaN,"826 Terrace Lake Drive, Aurora IL","(41.7367382, -88.2498821)",41.736738,-88.249882,POINT (-88.24988 41.73674),NaN
3,Germain House,"925 West Chicago Avenue, Chicago",Rockwell Property,Emmett Cunnigham Jr. and Sharon Shaw,$18 million,gain,Sales,green,NaN,"925 West Chicago Avenue, Chicago IL","(41.8960354, -87.651032)",41.896035,-87.651032,POINT (-87.65103 41.89604),NaN
4,The Aberdeen West Loop,"20 North Aberdeen Street, Chicago",Waterton,JAB Real Estate,$35 million,loss,Sales,red,NaN,"20 North Aberdeen Street, Chicago IL","(41.8823579, -87.65470839999999)",41.882358,-87.654708,POINT (-87.65471 41.88236),NaN
5,Paragon Chicago,"1326 South Michigan Avenue, Chicago",John Murphy and CIM Group,FPA Multifamily,$144 million,loss,Sales,red,NaN,"1326 South Michigan Avenue, Chicago IL","(41.8649257, -87.62449289999999)",41.864926,-87.624493,POINT (-87.62449 41.86493),NaN
6,David Gassman's Northside portfolio,"6954 North Sheridan Road, Chicago*",David Gassman,multiple buyers,$72 million,NaN,Sales,green,https://therealdeal.com/chicago/2024/04/03/dav...,"6954 North Sheridan Road, Chicago* IL","(42.00830029999999, -87.6622091)",42.008300,-87.662209,POINT (-87.66221 42.00830),"<a href=""https://therealdeal.com/chicago/2024/..."
7,850 North Michigan,"850 North Michigan Avenue, Chicago",JPMorgan Asset Management,Crescent Heights,$80 million,loss,Sales,red,NaN,"850 North Michigan Avenue, Chicago IL","(41.8983126, -87.624648)",41.898313,-87.624648,POINT (-87.62465 41.89831),NaN
8,Sixteen30 Apartments,"14750 Wallin Drive, Plainfield",Wingspan Development Group,Continental Properties,$78 million,gain,Sales,green,NaN,"14750 Wallin Drive, Plainfield IL","(41.6134211, -88.2199963)",41.613421,-88.219996,POINT (-88.22000 41.61342),NaN
9,Willow Crossing Apartments,"1031 Charlela Lane, Elk Grove Village",JPMorgan Asset Management,R.I.G. Capital,$89.3 million,gain,Sales,green,NaN,"1031 Charlela Lane, Elk Grove Village IL","(41.9947864, -88.0276456)",41.994786,-88.027646,POINT (-88.02765 41.99479),NaN


In [10]:
# Convert all data to strings if possible, or remove complex structures
for col in gdf.columns:
    if gdf[col].dtype == object:
        try:
            gdf[col] = gdf[col].astype(str)
        except Exception as e:
            print(f"Could not convert column {col}: {e}")
            gdf.drop(columns=[col], inplace=True)


In [11]:
gdf['color'].value_counts()

color
cyan     17
green     9
red       4
white     3
Name: count, dtype: int64

In [12]:
gdf.to_file('output_sales_new_listings.geojson', driver='GeoJSON')

In [13]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/chicago_sales_new_listings_07_31_24


<a href="https://therealdeal.com/chicago/2024/04/03/david-gassman-sells-chicago-apartment-portfolio-for-72m/"> 6954 North Sheridan Road, Chicago</a>